# Examples of using analysis functionalities

Using discovery_utils analyses functionalities for investments data

Here, we'll find companies using their categories, but you can also use search results from the process shown in cybersec_search.ipynb

In [1]:
from discovery_utils.utils import (
    analysis_crunchbase,
    analysis,
    charts
)

In [ ]:
from discovery_utils.getters import crunchbase
CB = crunchbase.CrunchbaseGetter()

Let's find categories to use for selecting companies

In [ ]:
# Categories for cybersec
CB.find_similar_categories("cyber security", category_type="broad", n_results=10)

In [ ]:
# Categories for family and kids
CB.find_similar_categories("family", category_type="narrow", n_results=15)

Now get both types of companies and take the intersection to find those that match both categories

This might take a minute for the first time, as it downloads the data

In [ ]:
cybersec_df = CB.get_companies_in_categories(["Privacy and Security"], category_type="broad")
family_df = CB.get_companies_in_categories(["Family", "Parenting", "Children", "Teenagers"], category_type="narrow")

In [6]:
matching_ids = set(cybersec_df.id) & set(family_df.id)

In [ ]:
len(matching_ids)

You can check these companies by querying the ids

In [ ]:
matchings_orgs_df = CB.organisations_enriched.query("id in @matching_ids")
matchings_orgs_df[['name', 'homepage_url', 'short_description']]

Now get the funding rounds for the matching companies - you can specify what type of funding rounds you need

In [ ]:
# Check what type of funding rounds there are
CB.unique_funding_round_types

In [10]:
funding_rounds_df = (
    CB.select_funding_rounds(org_ids=matching_ids, funding_round_types=["angel", "pre_seed", "seed", "series_a"])
)

Now let's generate some basic time series

In [ ]:
ts_df = analysis_crunchbase.get_timeseries(matchings_orgs_df, funding_rounds_df, period='year', min_year=2014, max_year=2024)
ts_df

In [ ]:
fig = charts.ts_bar(
    ts_df,
    variable='raised_amount_gbp_total',
    variable_title="Raised amount, £ millions",
    category_column="_category",
)
charts.configure_plots(fig, chart_title="")

Let's look into breakdown of deal types

In [27]:
deals_df, deal_counts_df = analysis_crunchbase.get_funding_by_year_and_range(funding_rounds_df, 2014, 2024)
aggregated_funding_types_df = analysis_crunchbase.aggregate_by_funding_round_types(funding_rounds_df)

In [ ]:
deals_df

In [ ]:
analysis_crunchbase.chart_investment_types(aggregated_funding_types_df)

In [ ]:
analysis_crunchbase.chart_deal_sizes(deals_df)

In [ ]:
analysis_crunchbase.chart_deal_sizes_counts(deal_counts_df)